This notebook demonstrates how to create a spatialite database from GMR inversions. Spatialite was chosen as the format as it is compact, easy to set up and readable by various software including GIS and python. Much of the creation depends on two spreadsheets referenced in this notebok which contain metadata, key spatial information and map inversion directories.

Neil Symington
neil.symington@gs.gov.au

In [1]:
import shapely.wkb
import shapely.wkt
import os
import pandas as pd
# sqlite/spatialite
from sqlalchemy import create_engine, event, ForeignKey
from sqlalchemy import Column, Integer, String, Float, Date, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlite3 import dbapi2 as sqlite
from scipy.io import loadmat
import sqlite3
import numpy as np

In [3]:
DB_PATH = r"C:\GA\SNMR\EastKimberley_SNMR.sqlite"

if os.path.exists(DB_PATH):
        os.remove(DB_PATH)

        
engine = create_engine('sqlite:///' + DB_PATH, module=sqlite, echo=False)

# load spatialite extension for sqlite. make sure that mod_spatialite.dll is located in a folder that is in your 
# system path

# This will only work on windows computers and the 

os.environ['PATH'] = r'C:\Users\symin\mod_spatialite-NG-win-amd64\mod_spatialite-NG-win-amd64' + ';' + os.environ['PATH']

@event.listens_for(engine, 'connect')
def connect(dbapi_connection, connection_rec):
    dbapi_connection.enable_load_extension(True)
    dbapi_connection.execute('SELECT load_extension("mod_spatialite")')

# create spatialite metadata
print('creating spatial metadata...')
engine.execute("SELECT InitSpatialMetaData(1);")

creating spatial metadata...


In [4]:
Base = declarative_base()

class Sites(Base):
    __tablename__ = 'sites'
    table_id = Column(Integer, index=True, primary_key=True)
    field_id = Column("Field_ID", String(20))
    site_code = Column("Site_Code", String(40))
    mid_x = Column("mid_X", Float)
    mid_y = Column("mid_Y", Float)
    declination = Column("declination_angle", Float)
    inclination = Column("inclination_angle", Float)
    date = Column("date", Date)
    loop_width = Column("loop_width", Float)
    coil_type = Column("coil_type", String(20))
    geometry = Column(String)
      
    
class Acquisitions(Base):
    __tablename__ = 'acquisitions'
    table_id = Column(Integer, index=True, primary_key=True)
    pulse_sequence = Column("pulse_sequence", String(5))
    pulse_length = Column("pulse_length", Float)
    
    
    site_id = Column(Integer, ForeignKey('sites.table_id'))
    sites = relationship("Sites")

    
class Inverse_model_metadata(Base):
    __tablename__ = 'inverse_model_metadata'
    table_id = Column(Integer, index=True, primary_key=True)
    doi = Column('Depth_of_Investigation', Float)
    reg_factor = Column('reg_factor', Float)
    cond_profile = Column("conductive_earth_model", Boolean)
    inversion_software = Column("Inversion_software", String(50))
    
    
    acquisition_id = Column(Integer, ForeignKey('acquisitions.table_id'))
    acquisitions = relationship("Acquisitions")
    
class Inverse_models(Base):
    __tablename__ = 'inverse_models'
    table_id = Column(Integer, index=True, primary_key=True)
    depth_from = Column('Depth_from', Float)
    depth_to = Column('Depth_to', Float)
    mobile_water_content = Column("Mobile_water_content", Float)
    bound_water_content = Column("Bound_water_content", Float)
    total_water_content = Column("Total_water_content", Float)
    T1 = Column("T1", Float)
    T2 = Column("T2", Float)
    T2_star = Column("T2*", Float)
    frequency = Column("frequency", Float)
    phase = Column("phase", Float)
    
    site_id = Column(Integer, ForeignKey('sites.table_id'))
    sites = relationship("Sites")
    
    acquisition_id = Column(Integer, ForeignKey('acquisitions.table_id'))
    acquisitions = relationship("Acquisitions")

    
    inversion_id = Column(Integer, ForeignKey('inverse_model_metadata.table_id'))
    inversions = relationship("Inverse_model_metadata")



In [5]:
Sites.__table__

Table('sites', MetaData(bind=None), Column('table_id', Integer(), table=<sites>, primary_key=True, nullable=False), Column('Field_ID', String(length=20), table=<sites>), Column('Site_Code', String(length=40), table=<sites>), Column('mid_X', Float(), table=<sites>), Column('mid_Y', Float(), table=<sites>), Column('declination_angle', Float(), table=<sites>), Column('inclination_angle', Float(), table=<sites>), Column('date', Date(), table=<sites>), Column('loop_width', Float(), table=<sites>), Column('coil_type', String(length=20), table=<sites>), Column('geometry', String(), table=<sites>), schema=None)

In [6]:
Base.metadata.create_all(engine)

In [7]:
infile = r"C:\GA\SNMR\EK_SNMR_location_metadata.csv"

df = pd.read_csv(infile)

df['date'] =pd.to_datetime(df.date, format ="%d/%m/%Y")

df = df.sort_values(by='date').reset_index(drop=True)

df

,Field_ID,Site_Code,X,Y,bo_dec_angle_deg,bo_inc_angle_deg,date,loop_width,coil_type,geometry
0,MillProf,EKSNMR16091601,466505.9100,8316696.170,0.0,-43.700,2016-09-16,100,square,"POLYGON ((466296.2798914838 8316647.306548808,..."
1,saltflats_st1,EKSNMR16091701,468448.6660,8354121.638,0.0,-43.700,2016-09-17,100,square,"POLYGON ((468500.9411914838 8354169.254548808,..."
2,2pm,EKSNMR16091703,468863.3010,8347878.272,0.0,-43.600,2016-09-17,100,square,"POLYGON ((468914.1478914838 8347926.009548808,..."
3,7m,EKSNMR16091704,464216.0790,8350319.661,0.0,-43.700,2016-09-17,100,square,"POLYGON ((464250.9278914838 8350216.810548807,..."
4,6m,EKSNMR16091803,458571.4590,8345568.043,0.0,-43.600,2016-09-18,100,square,"POLYGON ((458622.3728914838 8345611.113548808,..."
5,strat_trans_st3,EKSNMR16091801,458155.4900,8345306.897,0.0,-43.600,2016-09-18,100,square,"POLYGON ((458171.9589914838 8345485.717548807,..."
6,strat_trans_st2,EKSNMR16091804,457781.0190,8345010.551,0.0,-43.600,2016-09-18,100,square,"POLYGON ((457794.8020914838 8345219.131548807,..."
7,4m,EKSNMR16091902,476976.7710,8318919.739,0.0,-43.600,2016-09-19,100,square,"POLYGON ((477026.6028914838 8318967.239548807,..."
8,centr_spat_st_2,EKSNMR16091903,475577.6060,8325315.362,0.0,-43.900,2016-09-19,100,square,"POLYGON ((475629.5678914838 8325364.449548807,..."
9,east_spat_cov_st1,EKSNMR16092001,491729.4930,8334923.608,0.0,-43.900,2016-09-20,100,square,"POLYGON ((491781.4858914838 8334970.876548807,..."


In [8]:
# Drop columns that are not needed in the table


df.rename(columns = {'X': 'mid_X', 'Y': 'mid_Y', 'bo_inc_angle_deg': 'inclination_angle',
                     'bo_dec_angle_deg': 'declination_angle'}, inplace = True)

In [9]:
# Bring in the data into a dataframe

infile = r"C:\GA\SNMR\EK_conductive_earth_inversion_spreadsheet.csv"
df_acquisition = pd.read_csv(infile)

df_acquisition.columns

Index(['Field_ID', 'pathname', 'filename', 'pulse_sequence', 'pulse_length',
       'detect_coil', 'coil_diameter', 'n_turns', 'coil_type',
       'bo_inc_angle_deg', 'bo_dec_angle_deg', 'z_max', 'estimate_rdp',
       'reg_factor', 'use_cond_profile', 'loadcondpath', 'loadcondname',
       'close_figs', 'save_pathname', 'save_filename', 'matfile', 'hdf5_file'],
      dtype='object')

In [10]:
# Now add the location index and find it using a join on the site id column

loc_id = -999 * np.ones(len(df_acquisition['Field_ID']),
                      dtype = np.int64)

for i , item in enumerate(df_acquisition['Field_ID'].values):
    # find the location index
    loc_id[i] = df[df['Field_ID'] == item].index[0]


df_acquisition = df_acquisition.assign(site_id = loc_id)

df_acquisition['pulse_length'].dtype

df_acquisition = df_acquisition.sort_values(by='site_id').reset_index(drop=True)

df_acquisition.columns

Index(['Field_ID', 'pathname', 'filename', 'pulse_sequence', 'pulse_length',
       'detect_coil', 'coil_diameter', 'n_turns', 'coil_type',
       'bo_inc_angle_deg', 'bo_dec_angle_deg', 'z_max', 'estimate_rdp',
       'reg_factor', 'use_cond_profile', 'loadcondpath', 'loadcondname',
       'close_figs', 'save_pathname', 'save_filename', 'matfile', 'hdf5_file',
       'site_id'],
      dtype='object')

In [13]:

df_acquisition['inversion_file'] = ''

for index, row in df_acquisition.iterrows():
    new_path = row['save_pathname'].replace(r'D:\EastKimberley_SNMR\SNMR_database', r'C:\GA\SNMR')
    
    df_acquisition['inversion_file'].iloc[index] = new_path + row['save_filename'] + '\\' + row['save_filename'] + '_1d_inversion.txt'



C:\Users\symin\Anaconda3\envs\hydrogeol_utils\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [14]:
df_acquisition['inversion_file'].iloc[0]

'C:\\GA\\SNMR\\Inversions\\MillProf\\conductive_earth_inversion_FID_40ms\\conductive_earth_inversion_FID_40ms_1d_inversion.txt'

In [15]:
# Define headers for the various inversion files

header_dict  = {}

header_dict['FID'] = ['Depth_from', 'Depth_to', 'K_rel', 'water_content', 'T2*', 'frequency',
          'phase', 'T1', 'Bound_water_content', 'Mobile_water_content', 'Total_water_content']

header_dict['CPMG'] = ['Depth_from', 'Depth_to', 'K_rel', 'Mobile_water_content', 'T2']

header_dict['T1'] = ['Depth_from', 'Depth_to', 'K_rel', 'water_content', 'T2*', 'frequency',
          'phase', 'T1', 'Bound_water_content', 'Mobile_water_content', 'Total_water_content']

# This function parses the text file automatically created by the inversion algorithm


def parse_inversion_file(infile, pulse_sequence):
    
    '''
    infile: the filename of the text file
    sequence: ['FID', 'CPMG', 'T1']
    '''
    # Get header based on the sequence
    
    header = header_dict[pulse_sequence]
    
    #import data into numpy array
    data = np.loadtxt(infile, usecols = np.arange(0,len(header)))
    
    # Check if final row is just zeros
    if data[-1,0] == 0.:
        data = data[:-1,:]
    
    # Create a pandas dataframe
    df_inv = pd.DataFrame(data, columns = header)
    
    # Add a null column of T2* (FID and T1) or T2 (CPMG)
    #if 'T2*' not in header:
    #    df_inv['T2*'] = np.nan*np.ones(df_inv.shape[0])
    #elif 'T2' not in header:
    #    df_inv['T2'] = np.nan*np.ones(df_inv.shape[0])
    
    
    return df_inv

In [16]:
# Function to get the index of acquisition entry given site, and  acquisition parameters

def get_foreign_keys(df_acquisition, Field_ID, pulse_sequence, pulse_length):
    # Define criterion    
    criterion1 = df_acquisition['Field_ID'].map(lambda x: x== Field_ID)
    criterion2 = df_acquisition['pulse_sequence'].map(lambda x: x== pulse_sequence)
    criterion3 = df_acquisition['pulse_length'].map(lambda x: x == pulse_length)

    acquisition = df_acquisition[criterion1][criterion2][criterion3]
    
    return acquisition.site_id, acquisition.index

# Function for updating the acquisition metadata dataframe

def update_inversion_metadata(inv_id, acqu_ind, doi, 
                              reg_parameter, cond_profile):
    # First we convert the cond profile to a boolean
    
    if cond_profile == 0:
        conductive_earth = False
    else:
        conductive_earth = True
    
    d = {'inversion_id': inv_id,
         'acquisition_id': acqu_ind, 
         'DOI': doi,
         'conductive_earth': conductive_earth,
         'regularisation_parameter': reg_parameter}

    # Now add an entry to the df_invmet dataframe

    df_temp = df_invmet.append(pd.DataFrame.from_dict([d]),
                                ignore_index = True)
    return df_temp
    

In [17]:
# Create a table for inversion metadata

df_invmet = pd.DataFrame(columns =  ['inversion_id', 'acquisition_id', 'DOI',
                                     'conductive_earth', 'regularisation_parameter'])

In [18]:
# Create an empty dataframe into which to add data

df_inversion = pd.DataFrame(columns = ['inversion_id', 'Depth_from', 'Depth_to', 'K_rel', 
                                       'water_content', 'T2*','T2', 'frequency', 'phase', 'T1',
                                       'Bound_water_content', 'Mobile_water_content', 
                                       'Total_water_content', 'acquisition_id', 'site_id'])
inv_id = 0

# Now we import the inversion data

for index, row in df_acquisition.iterrows():
    
    infile = row['inversion_file']
    
    try:
        df_inv = parse_inversion_file(infile, row.pulse_sequence)
    
        site_ind, acqu_ind = row['site_id'], index
    
        reg_parameter = row['reg_factor']
        cond_profile = row['use_cond_profile']
    
    
        ##TODO fix this
        # Extract the depth of investigation from the matlab header
        fig = loadmat('\\'.join(infile.split('\\')[:-1]) + '\\resmatrix.fig')
    
        # Rather ugly hack to extract the depth of investigation from a 
        # .fig file

        doi = fig['hgS_070000'][0][0][3][1][0][3][1][0][2][0][0][7][0][0]
    
        # Update the inversion metadata dataframe
                
        df_invmet = update_inversion_metadata(inv_id, acqu_ind,
                                          doi, reg_parameter, cond_profile)
    

        df_inv['acquisition_id'] = int(acqu_ind)
        df_inv['site_id'] = int(site_ind)
        df_inv['inversion_id'] = inv_id
        
        df_inversion = df_inversion.append(df_inv)
        inv_id += 1
        
    except OSError:
        
        print(row.Field_ID)

    



C:\Users\symin\Anaconda3\envs\hydrogeol_utils\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


east_spat_cov_st1
SWI_VAL_St6
bon_east_12
bon_east_12
OB18_03
OB18_03
OB18_05
OK18_10


In [19]:
df_invmet.set_index('inversion_id', inplace = True)

In [20]:
# Now we try to get the dataframes into the database classes

all_sites = []

for index, row in df.iterrows():
    site = Sites(table_id = index,
                 field_id = row["Field_ID"], 
                site_code = row["Site_Code"], 
                mid_x = row["mid_X"], mid_y = row["mid_Y"],
                declination = row["declination_angle"],
                inclination = row["inclination_angle"],
                date = pd.to_datetime(row['date'], format = '%d/%m/%Y').date(),
                loop_width = row["loop_width"],
                coil_type = row["coil_type"],
                 geometry = row['geometry'])
    all_sites.append(site)


In [21]:
# Now we try to get the dataframes into the database classes

all_acquisitions = []

for index, row in df_acquisition.iterrows():
    acquisition = Acquisitions(table_id = index,
                               pulse_sequence = row['pulse_sequence'],
                               pulse_length = row['pulse_length'],
                               site_id = row['site_id'])
                               
    all_acquisitions.append(acquisition)


In [22]:
all_inversion_metadata = []

for index, row in df_invmet.iterrows():
    inv_met = Inverse_model_metadata(doi = row['DOI'],
                                     reg_factor = row['regularisation_parameter'],
                                     cond_profile = True,
                                     inversion_software = 'GMRInversion1D_CLI_v2.7.3',
                                     acquisition_id = row['acquisition_id'],
                                     table_id = index)
    all_inversion_metadata.append(inv_met)
                                     

In [23]:

all_inversions = []

for index, row in df_inversion.iterrows():
    inversion = Inverse_models(depth_from = row['Depth_from'],
                           depth_to = row['Depth_to'],
                           mobile_water_content = row['Mobile_water_content'],
                           bound_water_content = row["Bound_water_content"],
                           total_water_content =row["Total_water_content"],
                           T1 = row["T1"],
                           T2 = row["T2"],
                           T2_star = row["T2*"],
                           frequency = row["frequency"],
                           phase = row["phase"],
                           site_id = row['site_id'],
                           acquisition_id = row['acquisition_id'],
                              inversion_id = row['inversion_id'])
                               
    all_inversions.append(inversion)


In [24]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [25]:
session.add_all(all_sites)
session.add_all(all_acquisitions)
session.add_all(all_inversions)
session.add_all(all_inversion_metadata)

session.commit()

In [26]:
# add a Spatialite geometry column called 'geom' to the table, using ESPG 28352,
# data type POLYGON and 2 dimensions (x, y)
engine.execute("SELECT AddGeometryColumn('sites', 'geom', 28352, 'POLYGON', 2);")

# update the yet empty geom column by parsing the well-known-binary objects from the geometry column into 
# Spatialite geometry objects
engine.execute("UPDATE sites SET geom=GeomFromText(geometry, 28352);")

